 # Tutorial 07 - Lines and Corners
 
 ## Dr. David C. Schedl

 Note: this tutorial is geared towards students **experienced in programming** and aims to introduce you to **Digital Imaging / Computer Vision** techniques.


# Table of Contents  


- Hough Transformation
- Corner Detection


# Initilization

As always let's import useful libraries, first.

In [ ]:
import os
import cv2 # openCV
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.optimize import curve_fit
from ipywidgets import interact, fixed, IntSlider, FloatSlider

We will work with images today. So let's download some with `curl` (the same sources as in `02_OpenCV.ipynb`).

In [ ]:
!curl -o "cat.jpg" "https://raw.githubusercontent.com/Digital-Media/cv_data/main/example_images/cat.jpg" --silent
!curl -o "gogh.jpg" "https://raw.githubusercontent.com/Digital-Media/cv_data/main/example_images/gogh.jpg" --silent
!curl -o "sudoku.png" "https://raw.githubusercontent.com/Digital-Media/cv_data/main/example_images/sudoku.png" --silent
!curl -o "shapes.png" "https://raw.githubusercontent.com/Digital-Media/cv_data/main/example_images/shapes.png" --silent
!curl -o "woman.jpg" "https://raw.githubusercontent.com/Digital-Media/cv_data/main/example_images/woman.jpg" --silent

# Naive Line Fitting

Let's look at a simple example of line fitting, where we try to fit a line with a simple line equation: $y = mx + b$.
We use the `scipy.optimize` package to fit the line to the data. <br>
Note that this will only work for a single line and breaks if there are multiple lines or noise in the data.

In [ ]:
image = np.zeros((50,50),dtype=np.uint8)
image[3:33,10:40] = np.eye(30)*255
# let's add random noise (off if N=0)
N = 0
image[np.random.randint(0,50,N), np.random.randint(0,50,N)] = 255


# get all the non-zero points
points = np.argwhere(image)
ys, xs = points[:,0], points[:,1]

# a simple line equation y = mx + b (m is the slope, which you might also know as k)
def line_eq(x, m, b):
    return m*x + b

# find m and b
(m,b), _ = curve_fit(line_eq, xs, ys)
print(m,b)


#yshat = line_eq(xs, m, b)

# show
plt.imshow(image, cmap='gray')
#plt.plot(xs, line_eq(xs, m, b))
plt.show()

# Hough Transformation

## OpenCV's Hough implementation

Let's first look at the implementation available with OpenCV. 
We can reuse the edge image (`edges`) that we computed before.
The function call is
```python
cv2.HoughLines( edges, rho, theta, threshold )
```
and the parameters are the edge image, the distance resolution of the accumulator for $d$ in pixels and the angle $\theta$ in radians, and the accumulator threshold.

In [ ]:
img = cv2.imread('sudoku.png',0)

# OpenCVs implementation using the image as input (not the edges)
edges = cv2.Canny(img, 100, 200)
image = cv2.rotate( edges, cv2.ROTATE_90_CLOCKWISE ) # reuse edges image

# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# Standard Hough Line Transform
lines = cv2.HoughLines(image, 1, np.pi / 360, 150)

if lines is not None:
    for i in range(0, len(lines)):
        d = lines[i][0][0]
        theta = lines[i][0][1]

        #print("Line {:2d}: d={:6.2f}, theta={:3.0f}".format(i, d, np.rad2deg(theta)))

        # Plot the lines
        def line_equation(x):
            if abs(theta) < 0.01:
                x = int(np.cos(theta) * d)
                return x, 0 if x <= 0  else image.shape[0]-1

            return x, int((d - np.cos(theta) * x) / np.sin(theta))
        
        pt1 = line_equation(0)
        pt2 = line_equation(image.shape[1]-1)
        cv2.line( cdst, pt1, pt2, (255,0,0), 3, cv2.LINE_AA)

plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(image, cmap='gray')
plt.subplot(122), plt.imshow(cdst) #"Detected Lines (Todo)", 
plt.show()

## Probabilistic Hough Transform

OpenCV also provides a probabilistic Hough transform, which is faster for large images and allows to find lines with a minimum length. The function call is:
```python
cv2.HoughLinesP( edges, rho, theta, threshold, lines, minLineLength, maxLineGap )
``` 

In [ ]:
img = cv2.imread('sudoku.png',0)

# OpenCVs implementation using the image as input (not the edges)
edges = cv2.Canny(img, 100, 200)
image = cv2.rotate( edges, cv2.ROTATE_90_CLOCKWISE ) # reuse edges image
# optionally, we can use the probabilistic hough transform 
# which allows to set the minimum length of the line
lines = cv2.HoughLinesP(image, 1, np.pi / 360, 100, None, 100, 30)

# Copy edges to the images that will display the results in BGR
cdst = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for line in lines:
        x1,y1,x2,y2 = line[0]
        cv2.line( cdst, (x1,y1), (x2,y2), (255,0,0), 3, cv2.LINE_AA)

plt.figure(figsize=(20,10))
plt.subplot(121), plt.imshow(image, cmap='gray')
plt.subplot(122), plt.imshow(cdst) #"Detected Lines (Todo)", 
plt.show()

## Non-OpenCV implementation

To understand the algorithm better let's also look at an implementation without the builtin function of OpenCV. This implementation can also be found [online](https://github.com/alyssaq/hough_transform). 
Note: the implementation is only for illustration and there might be more efficient/robust ways of implementing it.

In [ ]:
def hough_line(img):
  # ds and Theta ranges
  thetas = np.deg2rad(np.arange(-90.0, 90.0))
  width, height = img.shape
  diag_len = int(np.ceil(np.sqrt(width * width + height * height)))  # max_dist
  ds = np.linspace(-diag_len, diag_len, diag_len*2)

  # Cache some resuable values
  cos_t = np.cos(thetas)
  sin_t = np.sin(thetas)
  num_thetas = len(thetas)

  # Hough accumulator array of theta vs rho
  accumulator = np.zeros((2 * diag_len, num_thetas), dtype=np.uint64)
  y_idxs, x_idxs = np.nonzero(img)  # (row, col) indexes to edges

  # Vote in the hough accumulator
  for i in range(len(x_idxs)):
    x = x_idxs[i]
    y = y_idxs[i]

    for t_idx in range(num_thetas):
      # Calculate rho. diag_len is added for a positive index
      d = round(x * cos_t[t_idx] + y * sin_t[t_idx]) + diag_len
      accumulator[d, t_idx] += 1

  return accumulator, thetas, ds

# Create binary image and call hough_line
image = np.zeros((50,50),dtype=np.uint8)
#image[:, :] = np.eye(50)[::-1, :]# + np.eye(50)#[::-1, :]
image[3:33,10:40] = np.eye(30)*255
# let's add random noise (off if N=0)
N = 0
image[np.random.randint(0,50,N), np.random.randint(0,50,N)] = 255

accumulator, thetas, ds = hough_line(image)

cdst = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)


# Easiest peak finding based on max votes
idx = np.argmax(accumulator)
d = ds[int(idx / accumulator.shape[1])]
theta = thetas[idx % accumulator.shape[1]]
print("d={:.2f}, theta={:.0f}".format(d, np.rad2deg(theta)))

# Plot the line
def line_equation(x):
    if abs(theta) < 0.01:
        x_ = int(d)
        return x_, 0 if x <= 0  else image.shape[0]-1

    return x, int((d - np.cos(theta) * x) / np.sin(theta))

pt1 = line_equation(0)
pt2 = line_equation(image.shape[1]-1)
cv2.line( cdst, pt1, pt2, (255,0,0), 1, cv2.LINE_AA)

plt.figure(figsize=(15,10))
plt.subplot(121), plt.imshow(cdst), plt.title( 'original' )
plt.subplot(122), plt.imshow((accumulator), cmap='hot'), plt.title( 'Hough space' ) 
plt.show()

# Harris Corner Detection

Let's display the result of the Harris corner detection algorithm. The function call is:
```python
cv2.cornerHarris( gray, blockSize, ksize, k )
```
where `gray` is the grayscale image, `blockSize` is the size of the neighborhood considered for corner detection, `ksize` is the aperture parameter of the Sobel derivative used, and `k` is a free parameter of the Harris detector.

In [ ]:
img = cv2.imread('sudoku.png',0)

def corner_harris(block_size, ksize, k):
    # OpenCVs implementation using the image as input (not the edges)
    corners = cv2.cornerHarris(img, blockSize=block_size, ksize=ksize, k=k)

    # Copy edges to the images that will display the results in BGR
    cdst = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # Threshold for an optimal value, it may vary depending on the image.
    cdst[corners>0.01*corners.max()]=[0,0,255]

    plt.figure(figsize=(10,10))
    plt.imshow(cdst) #"Detected Edges",
    plt.show()

# interactivity
interact(
    corner_harris,
    block_size=IntSlider(min=1, max=50, step=1, value=5),
    ksize=IntSlider(min=1, max=9, step=2, value=3),
    k = FloatSlider(min=0.01, max=0.5, step=0.01, value=0.04)
)